In [1]:
import pandas as pd
import numpy as np

In [2]:
fifa = pd.read_csv('/data/jhan0801/thec03u5/fifa-18-demo-player-dataset/CompleteDataset.csv')
fifa_with_league = pd.read_csv('/data/jhan0801/kevinmh/fifa-18-more-complete-player-dataset/complete.csv')

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (23,35) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
fifa.head()

,Unnamed: 0,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,...,RB,RCB,RCM,RDM,RF,RM,RS,RW,RWB,ST
0,0,Cristiano Ronaldo,32,https://cdn.sofifa.org/48/18/players/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Real Madrid CF,https://cdn.sofifa.org/24/18/teams/243.png,...,61.0,53.0,82.0,62.0,91.0,89.0,92.0,91.0,66.0,92.0
1,1,L. Messi,30,https://cdn.sofifa.org/48/18/players/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,93,93,FC Barcelona,https://cdn.sofifa.org/24/18/teams/241.png,...,57.0,45.0,84.0,59.0,92.0,90.0,88.0,91.0,62.0,88.0
2,2,Neymar,25,https://cdn.sofifa.org/48/18/players/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,94,Paris Saint-Germain,https://cdn.sofifa.org/24/18/teams/73.png,...,59.0,46.0,79.0,59.0,88.0,87.0,84.0,89.0,64.0,84.0
3,3,L. Suárez,30,https://cdn.sofifa.org/48/18/players/176580.png,Uruguay,https://cdn.sofifa.org/flags/60.png,92,92,FC Barcelona,https://cdn.sofifa.org/24/18/teams/241.png,...,64.0,58.0,80.0,65.0,88.0,85.0,88.0,87.0,68.0,88.0
4,4,M. Neuer,31,https://cdn.sofifa.org/48/18/players/167495.png,Germany,https://cdn.sofifa.org/flags/21.png,92,92,FC Bayern Munich,https://cdn.sofifa.org/24/18/teams/21.png,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Drop unnecessary data and Add in appropriate data

There are multiple data that we are not going to use such as club_logo, birth_date, real_face, flag etc. Since there are more data to drop, we will just select which data we want to use.

Also we want to use multiple different dataframe so we can use them when we explore the data and when we actually predict the data.

we will make:

 - `player analysis data` basic player's info
 - `prediction data` includes player's attributes, age, and position preferences

In [4]:
#drops all unnecessary data and merge data you need from other csv files.
analysis_columns = [
    'ID',
    'Name',
    'Club',
    'Age',
    'Nationality',
    'Overall',
    'Potential',
    'Value',
    'Wage',
    'Preferred Positions'
]

fifa_player_analysis = pd.DataFrame(fifa, columns=analysis_columns)
fifa_league = fifa_with_league[['ID', 'league']]
fifa_player_analysis = pd.merge(fifa_league, fifa_player_analysis, how='inner', on='ID')
fifa_player_analysis.head()

,ID,league,Name,Club,Age,Nationality,Overall,Potential,Value,Wage,Preferred Positions
0,20801,Spanish Primera División,Cristiano Ronaldo,Real Madrid CF,32,Portugal,94,94,€95.5M,€565K,ST LW
1,158023,Spanish Primera División,L. Messi,FC Barcelona,30,Argentina,93,93,€105M,€565K,RW
2,190871,French Ligue 1,Neymar,Paris Saint-Germain,25,Brazil,92,94,€123M,€280K,LW
3,176580,Spanish Primera División,L. Suárez,FC Barcelona,30,Uruguay,92,92,€97M,€510K,ST
4,167495,German Bundesliga,M. Neuer,FC Bayern Munich,31,Germany,92,92,€61M,€230K,GK


Let's add their position by their first preferred positions.

In [5]:
fifa_player_analysis['Position'] = fifa_player_analysis['Preferred Positions'].str.split().str[0]

Since value and wages are combination of string and number, we will change it to pure number

In [6]:
#We change M to million and K to thousand. if there is nothing, we just return the float number of it.
#Since there are Euro sign in front of the value, we start from 1 instead of 0
def strToNum(amount):
    if amount[-1] == 'M':
        return float(amount[1:-1])*1000000
    elif amount[-1] == 'K':
        return float(amount[1:-1])*1000
    else:
        return float(amount[1: ])
    

In [7]:
assert strToNum('E423M') == 423000000.0
assert strToNum('E12K') == 12000.0
assert strToNum('E12') == 12

In [8]:
fifa_player_analysis['numValue'] = fifa_player_analysis['Value'].apply(lambda v: strToNum(v))
fifa_player_analysis['numWage'] = fifa_player_analysis['Wage'].apply(lambda w: strToNum(w))
fifa_player_analysis.head()

,ID,league,Name,Club,Age,Nationality,Overall,Potential,Value,Wage,Preferred Positions,Position,numValue,numWage
0,20801,Spanish Primera División,Cristiano Ronaldo,Real Madrid CF,32,Portugal,94,94,€95.5M,€565K,ST LW,ST,95500000.0,565000.0
1,158023,Spanish Primera División,L. Messi,FC Barcelona,30,Argentina,93,93,€105M,€565K,RW,RW,105000000.0,565000.0
2,190871,French Ligue 1,Neymar,Paris Saint-Germain,25,Brazil,92,94,€123M,€280K,LW,LW,123000000.0,280000.0
3,176580,Spanish Primera División,L. Suárez,FC Barcelona,30,Uruguay,92,92,€97M,€510K,ST,ST,97000000.0,510000.0
4,167495,German Bundesliga,M. Neuer,FC Bayern Munich,31,Germany,92,92,€61M,€230K,GK,GK,61000000.0,230000.0


We will drop the goal keepers since it is too obvious to figure out the position.

In [9]:
prediction_columns = [
    'ID',
    'Aggression','Crossing', 'Curve', 'Dribbling', 'Finishing', 'Free kick accuracy', 
    'Heading accuracy', 'Long shots','Penalties', 'Shot power', 'Volleys', 'Short passing', 
    'Long passing', 'Interceptions', 'Marking', 'Sliding tackle', 'Standing tackle',
    'Strength', 'Vision', 'Acceleration', 'Agility', 'Reactions', 'Stamina', 'Balance', 
    'Ball control','Composure','Jumping', 'Sprint speed', 'Positioning',
    'Preferred Positions'
]

fifa_predict_position = pd.DataFrame(fifa, columns=prediction_columns)
fifa_predict_position['Preferred Positions'] = fifa_player_analysis['Position']
fifa_predict_position = fifa_predict_position[fifa_predict_position['Preferred Positions'] != 'GK']

fifa_predict_position.iloc[::2000, :]

,ID,Aggression,Crossing,Curve,Dribbling,Finishing,Free kick accuracy,Heading accuracy,Long shots,Penalties,...,Agility,Reactions,Stamina,Balance,Ball control,Composure,Jumping,Sprint speed,Positioning,Preferred Positions
0,20801,63,85,81,91,94,76,88,92,85,...,89,96,92,63,93,95,95,91,95,ST
2224,3665,58,69,73,70,73,71,58,75,76,...,60,73,58,71,79,82,32,54,67,LM
4411,229758,54,59,70,76,50,55,46,68,58,...,72,66,72,60,78,63,57,70,64,CAM
6617,148210,60,65,70,67,68,61,66,55,52,...,63,62,64,66,73,67,58,79,69,CB
8820,201484,46,58,64,74,62,59,56,63,61,...,80,59,48,66,64,61,54,78,62,LB
11041,190239,45,54,30,63,62-1,31,63,59,55,...,81,57,72,56,60,60-1,67,81,60,ST
13306,237192,49,62,60,62,53,58,42,63,59,...,67,62,66,69,67,55,59,68,56,CB
15592,223762,48,54,42,43,38,39,51,36,42,...,57,52,63,56,49,49,50,66,45,ST


In columns, there are few value that has attributes written in form like '70 + 9'

In [10]:
#changes + to - as it goes through the dataset.
def getNumbers(val):
    res = str(val)
    if '+' in res:
        fir, sec  = res.split('+')
        res = int(fir)+int(sec)
    elif '-' in res:
        fir, sec = res.split('-')
        if len(fir)== 0: res = -1
        else: res = int(fir)-int(sec)
    elif res == '':
        res = 0
    return float(res)

In [11]:
assert getNumbers('65 + 5') == 70
assert getNumbers('65 - 5') == 60
assert getNumbers('65') == 65

In [12]:
for i in range (1,30):
    fifa_predict_position.iloc[: ,i] = \
     fifa_predict_position.iloc[:, i].apply(lambda val: getNumbers(val))

In [13]:
fifa_predict_position.Crossing.dtype

dtype('float64')

In [14]:
fifa_predict_position.iloc[::2000, :]

,ID,Aggression,Crossing,Curve,Dribbling,Finishing,Free kick accuracy,Heading accuracy,Long shots,Penalties,...,Agility,Reactions,Stamina,Balance,Ball control,Composure,Jumping,Sprint speed,Positioning,Preferred Positions
0,20801,63.0,85.0,81.0,91.0,94.0,76.0,88.0,92.0,85.0,...,89.0,96.0,92.0,63.0,93.0,95.0,95.0,91.0,95.0,ST
2224,3665,58.0,69.0,73.0,70.0,73.0,71.0,58.0,75.0,76.0,...,60.0,73.0,58.0,71.0,79.0,82.0,32.0,54.0,67.0,LM
4411,229758,54.0,59.0,70.0,76.0,50.0,55.0,46.0,68.0,58.0,...,72.0,66.0,72.0,60.0,78.0,63.0,57.0,70.0,64.0,CAM
6617,148210,60.0,65.0,70.0,67.0,68.0,61.0,66.0,55.0,52.0,...,63.0,62.0,64.0,66.0,73.0,67.0,58.0,79.0,69.0,CB
8820,201484,46.0,58.0,64.0,74.0,62.0,59.0,56.0,63.0,61.0,...,80.0,59.0,48.0,66.0,64.0,61.0,54.0,78.0,62.0,LB
11041,190239,45.0,54.0,30.0,63.0,61.0,31.0,63.0,59.0,55.0,...,81.0,57.0,72.0,56.0,60.0,59.0,67.0,81.0,60.0,ST
13306,237192,49.0,62.0,60.0,62.0,53.0,58.0,42.0,63.0,59.0,...,67.0,62.0,66.0,69.0,67.0,55.0,59.0,68.0,56.0,CB
15592,223762,48.0,54.0,42.0,43.0,38.0,39.0,51.0,36.0,42.0,...,57.0,52.0,63.0,56.0,49.0,49.0,50.0,66.0,45.0,ST


Perfect! We now have a float data type, not python object

## Fill any empty column

There are some columns missing. Fill NaN with appropriate value and check if there is any null value in the data

In [15]:
fifa_predict_position = fifa_predict_position.dropna()

fifa_predict_position.isnull().values.any()

False

In [16]:
fifa_player_analysis['Club'] = fifa_player_analysis['Club'].fillna(value="None")
fifa_player_analysis['league'] = fifa_player_analysis['league'].fillna(value="None")
fifa_player_analysis.isnull().values.any()

False

## Write the data

Now that our table are cleaned up, we will now write the dataframe to files so we can access them in the next notebook

In [17]:
fifa_player_analysis.to_pickle('fifa_player_analysis')
fifa_predict_position.to_pickle('fifa_predict_position')